In [1]:
import pandas as pd
import seaborn as sns 
import pandas as pd
import numpy as np
from random import sample

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

## Random Forest

O Random Forest é uma técnica de aprendizado em conjunto, assim como o Bagging, que é utilizada para tarefas de classificação, regressão e outras. Seu funcionamento é baseado na criação de diversas árvores de decisão, lidando com o problema de overfit ao se criar uma única árvore de decisão com grande profundidade.

Dessa forma, o Random Forest é uma técnica útil para lidar com problemas de overfitting e para melhorar a performance de modelos de aprendizado de máquina em geral. Além disso, ele apresenta a vantagem de ser relativamente fácil de implementar e de ser capaz de lidar com uma grande variedade de tipos de dados.

## Passo a Passo e Implementação

### Bootstrap + feature selection

Dada uma base de dados, realizaremos uma amostragem com reposição (bootstrap) e geraremos novas bases uniformes, aleatórias e com reposição. Sendo a quantidade e tamanho das bases a ser definidas de antemão, do mesmo modo que no bagging.

O algoritmo de treinamento para a Random Forest aplica a técnica geral do Bootstrap com uma pequena modificação dividindo também as colunas com as variáveis explicativas do modelo (feature selection). Respeitando a regra:  m novas bases de dados com raiz quadrada de p colunas de variáveis explicativas para um problema de classificação ou p colunas dividido por 3 de variáveis explicativas para um problema de regressão

In [2]:
df = sns.load_dataset('penguins')
df.dropna(inplace = True)
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male


In [3]:
# exclui uma das espécies para facilitar o entendimento
df.drop(df.query('species == "Chinstrap"').index, inplace = True)
df.species.value_counts()

Adelie    146
Gentoo    119
Name: species, dtype: int64

In [4]:
alt = {'Adelie' : 0, 'Gentoo' : 1}
df.species = df.species.map(alt)
df.species.value_counts()

0    146
1    119
Name: species, dtype: int64

In [5]:
# classificação m = np.sqrt(p) = 2

def bootsfeat(base, m, p):
    subdf_X = []
    subdf_y = []
    col = ['bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g']
    
    for i in range(0,int(len(base)/m)):
        subbase = base.sample(int(len(base)/m))
        X = subbase[sample(col, p)]
        y = subbase.species
        subdf_X.append(X)
        subdf_y.append(y)
                
    return subdf_y, subdf_X

In [6]:
subdf_y, subdf_X = bootsfeat(df, 35, 2)
subdf_X[1]

,bill_depth_mm,bill_length_mm
308,14.0,47.5
138,16.5,37.0
226,14.6,45.4
251,14.2,42.8
90,18.0,35.7
305,17.3,50.8
234,14.6,45.8


### Modelagem

Então escolheremos nosso modelo para classificação ou regressão, como por exemplo árveore de decisão. O modelo escolhido será aplicado para cada nova base gerada no passo anterior

In [7]:
# arvore de decisão

def modelagem(subdf_y, subdf_X):
    arvores = []
    for i in range(0,int(len(subdf_y))):
        clf = tree.DecisionTreeClassifier(random_state = 1234)
        clf = clf.fit(subdf_X[i], subdf_y[i])
        y_pred = clf.predict(subdf_X[i])
        arvores.append(y_pred)
                   
    return arvores

In [8]:
arvores = modelagem(subdf_y, subdf_X)
arvores

[array([0, 0, 0, 0, 1, 0, 0], dtype=int64),
 array([1, 0, 1, 1, 0, 1, 1], dtype=int64),
 array([1, 1, 1, 0, 0, 0, 1], dtype=int64),
 array([0, 1, 1, 0, 0, 0, 1], dtype=int64),
 array([0, 0, 0, 1, 0, 1, 0], dtype=int64),
 array([1, 0, 0, 0, 0, 0, 1], dtype=int64),
 array([0, 0, 1, 1, 1, 0, 0], dtype=int64)]

### Aggregating

Após ajustarmos os modelos na etapa anterior, combinamos os resultados por sua média (para modelos de regressão) ou por votação (para modelos de classificação).

In [9]:
def agrega(subdf_X, subdf_y, arvores):
    df_pred = pd.DataFrame()

    for i in range(0,int(len(subdf_X))):
        subdf_X[i]['original'] = subdf_y[i]
        subdf_X[i]['predito'] = arvores[i]

        for j in range(0,int(len(subdf_X))):

            df_pred = pd.concat([df_pred,subdf_X[j]])

    return df_pred.groupby(['original', 'predito'])[['original', 'predito']].count()


In [10]:
agrega(subdf_X, subdf_y, arvores)

,,original,predito
original,predito,,
0.0,0.0,114,114
1.0,1.0,82,82


## Diferença entre Bagging e Random Forest

O Bagging é uma técnica de ensemble que cria múltiplos modelos independentes, enquanto a Random Forest é uma variação do Bagging que usa árvores de decisão como modelo base e usa amostragem aleatória dos recursos e do conjunto de dados de treinamento para reduzir a correlação entre as árvores. Ambas as técnicas são eficazes em melhorar a precisão do modelo e reduzir o overfitting.